In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [2]:
# load data 
optiData = pd.read_csv(r'C:\Users\isaldiviagonzatti\Downloads\gitFiles\MScThesisCode\FLP\output\optiData.csv', index_col=0)

In [3]:
# define supply of PAL grouped by orig nodes. 
# Tonnes are the sum of all farms corresponding to node (note that mean is applied because of matrix form)
supplyPAL = pd.DataFrame(optiData.groupby('origs', as_index=False).agg(
                                            leavesTonne=('leavesTonne','mean'),
                                        )
                     )

In [4]:
# define number of PAL origs and facilities dests 
pal = list(set(optiData.origs))  # PAL points
fac = list(set(optiData.dests))  # candidate plant locations

In [5]:
optiData.head()

,origs,dests,path_length,osmidOrig,areaHa,leavesTonne,distance,totalDist,costTrans
0,3610149725,2656380182,157664.026,3610149725,15.49063,2540.463291,319.129599,157.983156,31.596631
1,3610149725,5294901777,83956.035,3610149725,15.49063,2540.463291,319.129599,84.275165,16.855033
2,3610149725,1321571077,201954.500,3610149725,15.49063,2540.463291,319.129599,202.273630,40.454726
3,3610149725,3677820110,75493.804,3610149725,15.49063,2540.463291,319.129599,75.812934,15.162587
4,3610149725,3510580697,21814.095,3610149725,15.49063,2540.463291,319.129599,22.133225,4.426645


In [6]:
# transport costs converted into dict of the form (dests, origs): cost for tonne per trip
costTrans = optiData.groupby(['dests','origs']).apply(lambda x: np.float64(x['costTrans'].values).round(2)).to_dict()

In [7]:
# The (annual) facility costs 3MM / 15 years + annual operating costs 3 M
fac_cost = { i : 500000 for i in fac }

# capacity of 170,428 tonnes/year
capacity = { i : 170428 for i in fac }

# supply of PAL from each (aggregated) farm
supply = dict(zip(supplyPAL.origs, np.int64(supplyPAL.leavesTonne)))

In [8]:
print('Example data (not related to each other):', 
      '\n PAL node:', pal[:1], 
      '\n Facility node:', fac[:1], 
      '\n Transport costs per tonne for specific trip:', next(iter( costTrans.items() )) ,
      '\n Facility costs:', next(iter( fac_cost.items() )) , 
      '\n Facility capacity:', next(iter( capacity.items() )) , 
      '\n Supply of PAL per node:', next(iter( supply.items() ))  
      ) 

Example data (not related to each other): 
 PAL node: [5060446209] 
 Facility node: [4950368263] 
 Transport costs per tonne for specific trip: ((183803630, 185483242), 16.94) 
 Facility costs: (4950368263, 500000) 
 Facility capacity: (4950368263, 170428) 
 Supply of PAL per node: (185483242, 3063)


In [9]:
# create model 
m = gp.Model()

# create variables, x[i,j] = number of units that PAL node j sends to facility i
x = m.addVars( fac, pal, vtype=GRB.INTEGER )

# create variables, y[i] = 1 if locate a facility at site i
y = m.addVars( fac, vtype=GRB.BINARY )

In [10]:
# Objective: minimize transportation cost (annual) + facility cost (annual)
m.setObjective( gp.quicksum( costTrans[i,j] * x[i,j] for i in fac for j in pal ) + gp.quicksum( fac_cost[i] * y[i] for i in fac) , GRB.MINIMIZE )

In [11]:
# Constraints: each palPoint point should send its full supply
m.addConstrs( gp.quicksum( x[i,j] for i in fac ) == supply[j] for j in pal )

# Constraints: if no factory is built at site i, it cannot receive any PAL
m.addConstrs( gp.quicksum( x[i,j] for j in pal ) <= capacity[i] * y[i] for i in fac )

# Optional strengthening constraints
m.addConstrs( x[i,j] <= supply[j] * y[i] for i in fac for j in pal )

m.update()

In [13]:
%%time

# force Gurobi to use concurrent method (primal simplex, dual simplex, barrier all at the same time!)
m.Params.Method = 3

m.setParam('MIPGap', 0.05)
m.setParam('Timelimit', 300)

# solve
m.optimize()

Set parameter MIPGap to value 0.05
Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 567161 rows, 565995 columns and 2262525 nonzeros
Model fingerprint: 0x84f23473
Variable types: 0 continuous, 565995 integer (485 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [4e-02, 5e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 1e+05]
Found heuristic solution: objective 4.177701e+08
Presolve time: 2.75s
Presolved: 567161 rows, 565995 columns, 2262525 nonzeros
Variable types: 0 continuous, 565995 integer (485 binary)
Found heuristic solution: objective 4.177599e+08
Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.49s

Barrier statisti

In [14]:
medians = [ i for i in fac if y[i].x > 0.5 ]


In [15]:
k = len(medians)
assignedL = []
for p in range(k):
    i = medians[p] 
    assigned = [ j for j in pal if x[i,j].x > 0.5 ]
    assignedL.append(assigned)

In [42]:
medians = pd.DataFrame(medians, columns=["medians"])
medians.to_csv('mediansOpti.csv', index=False)

In [44]:
assignedL = pd.DataFrame(assignedL)
assignedL.to_csv('assignedL.csv', index=False)

In [37]:
lookFac = optiData.set_index('dests').loc[medians]

,origs,path_length,osmidOrig,areaHa,leavesTonne,distance,totalDist,costTrans
dests,,,,,,,,
2683369498,3610149725,14108.014,3610149725,15.490630,2540.463291,319.129599,14.427144,2.885429
2683369498,2656380182,171789.902,2656380182,23.333674,3826.722562,552.280437,172.342182,34.468436
2683369498,6208897979,106230.810,6208897979,6.951003,1139.964454,502.902622,106.733713,21.346743
2683369498,9858660252,110312.396,9858660252,4.081478,669.362390,1133.950309,111.446346,22.289269
2683369498,5192570962,192099.162,5192570962,169.502448,27798.401439,3527.432179,195.626594,39.125319
...,...,...,...,...,...,...,...,...
5973237709,9188115220,71921.906,9188115220,138.200597,22664.897949,4092.593453,76.014499,15.202900
5973237709,4941255384,179888.034,4941255384,23.371858,3832.984728,967.962044,180.855996,36.171199
5973237709,1682508166,104434.035,1682508166,143.308737,23502.632885,838.025586,105.272061,21.054412
